## We'll extend our previous examples

We can now extend [05 - Include an IFrame.ipynb](05%20-%20Include%20an%20IFrame.ipynb) to pass data to the iframe using either the query string or using `window.postMessage`.  In this case we'll use `https://soasta.github.io/osbridge-julia-d3/d3/07-d3-external-data.html?data=<data>` as our URL.

Our `createIFrame` and `updateIFrame` functions remain the same.

In [7]:
function createIFrame()
    displayid = "demo-iframe-" * string(rand())
    
    display(
        "text/html", 
        """
    <!-- First create an empty iframe that's 500px high and has this id -->
    <iframe
        id="$(displayid)"
        height="500"
        style="border:none;"
        src="about:blank">
    </iframe>


    <!-- Next create a JavaScript function with the same name as the node -->
    <script>
        window["$(displayid)"] = function(url) {
            var iframe = document.getElementById("$(displayid)");
            if(iframe) {
                iframe.width = iframe.parentNode.offsetWidth * 0.98;
                iframe.src = url;
            }
        };
    </script>
        """
    )
    
    return displayid
end

createIFrame (generic function with 1 method)

### Add Type to `updateIFrame` signature

Notice that we add a String Type for the `updateIFrame` method.  This will allow us to overload it later

In [3]:
function updateIFrame(displayid::String, url::String)
    display(
        "text/html",
        """<script>window["$(displayid)"]("$(url)");</script>"""
    )
end

updateIFrame (generic function with 1 method)

## Add `window.postMessage`

We now overload the `updateIFrame` method to accept a Numeric Array and pass that on to the iframe using `window.postMessage`

In [29]:
function updateIFrame(displayid::String, data::Array{Any, 1})
    display(
        "text/html",
        """
        <script>
        (function() {
            var iframe = document.getElementById("$(displayid)");
            if(iframe) {
                iframe.contentWindow.postMessage($(JSON.json(data)), "*");
            }
        }())
        </script>
        """
    )
end

updateIFrame (generic function with 3 methods)

## Tying these together, we can draw and update the iframe

In [17]:
id = createIFrame()

# Note that we do not include the [] for data here since this is not JSON, it's just a csv
updateIFrame(id, "https://soasta.github.io/osbridge-julia-d3/d3/07-d3-external-data.html?data=36,90,168,370,589,867,951,873,838,752,637,542,479,359,303,291,199,186,146,119,115,100,77,90,67,753")

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

## We can pass new data to the IFrame

By calling `updateIFrame` with a a data array, we can change what's displayed

In [18]:
updateIFrame(id, Int64[0,0,0,2,6,5,10,11,15,18,25,11,19,8,4,8,3,10,6,6,2,2,1,1,1,24])

## DataFrame to IFrame

We can even get our DataFrame code to pass data to the IFrame

In [20]:
using DataFrames
df = readtable("data.csv");

# Function to set histogram thresholds after dropping outliers based on IQR
function getSymmetricThresholds(results::DataFrame; timer::Symbol=:timers_t_done)
    summary = summarystats(results[timer])
    fw  = (summary.q75-summary.q25)*1.5

    low = round(Int64, max(summary.min, summary.q25-fw))
    high = round(Int64, min(summary.max, summary.q75+fw))+1

    thresholds::Array{Int64, 1} = []

    nthresholds=25

    range = high - low

    for i in 0:nthresholds-1
        push!(thresholds, round(Int64, low + i * range/nthresholds))
    end

    push!(thresholds, high)
    if high < round(Int64, summary.max)
        push!(thresholds, round(Int64, summary.max))
    end

    return thresholds
end

getSymmetricThresholds (generic function with 1 method)

In [21]:
thresholds = getSymmetricThresholds(df)

groups = by(
    df,
    :user_agent_family, 
    rows -> DataFrame(
        count = size(rows, 1),
        median = median(rows[:timers_t_done]),
        hist = JSON.json(hist(rows[:timers_t_done], thresholds)[2])
    )
)

,user_agent_family,count,median,hist
1,AOL,5,7119.0,"[0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0]"
2,Android Browser,32,14062.0,"[0,0,0,0,1,0,0,0,1,1,1,2,0,0,1,0,0,2,1,0,1,1,0,0,0,20]"
3,BlackBerry WebKit,3,11658.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1]"
4,Chrome,3423,3961.0,"[15,39,81,174,283,367,371,311,309,255,208,161,135,100,89,75,69,52,27,27,28,16,20,21,12,178]"
5,Chrome Mobile,57,8059.0,"[1,0,0,0,0,0,0,1,2,2,1,4,2,0,6,8,2,1,5,1,1,0,3,2,3,12]"
6,Chrome Mobile iOS,3,8589.0,"[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1]"
7,Firefox,1609,4978.0,"[5,8,17,32,62,95,112,148,153,121,122,93,75,58,61,66,40,33,38,28,21,29,8,19,10,155]"
8,IE,3309,5047.0,"[6,25,34,75,136,234,270,253,243,268,226,214,204,152,122,110,69,68,50,46,54,48,37,37,36,291]"
9,IE Mobile,20,14156.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,2,2,1,12]"
10,Iceweasel,1,5657.0,"[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]"


In [27]:
groups[groups[:user_agent_family] .== "Firefox", :hist][1]

"[5,8,17,32,62,95,112,148,153,121,122,93,75,58,61,66,40,33,38,28,21,29,8,19,10,155]"

In [30]:
# Take the histogram (string) out of the dataframe, and convert it to a numeric array
groupdata = JSON.parse(groups[groups[:user_agent_family] .== "Firefox", :hist][1])


# Pass that array to the IFrame
updateIFrame(id, groupdata)

The output shows up in the iframe above, so scroll up to see it

In fact, we could draw multiple histograms, one for each of the groups that we created

In [46]:
for i in 1:size(groups, 1)
    local id = createIFrame()
    local data = groups[i, :hist]
    updateIFrame(id, "https://soasta.github.io/osbridge-julia-d3/d3/07-d3-external-data.html?data=$(data)")

    display("text/html", """
    <p style="border-top: dashed 1px #ddd; border-bottom: solid 2px #000; padding-bottom: 3em;">
        <strong>$(groups[i, :user_agent_family]):</strong>
        <em>$(groups[i, :count])</em> records, median load time: <em>$(groups[i, :median])</em>ms
        </p>
        """
    )
end

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

AOL: 
 5 records, median load time: 7119.0 ms

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

Android Browser: 
 32 records, median load time: 14062.0 ms

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

...

Opera: 
 19 records, median load time: 5841.0 ms

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

Pale Moon (Firefox Variant): 
 1 records, median load time: 6097.0 ms

<!-- First create an empty iframe that's 500px high and has this id -->
 
 


<!-- Next create a JavaScript function with the same name as the node -->

Safari: 
 1318 records, median load time: 3574.5 ms